In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import nltk
from nltk import ngrams
from nltk.tokenize import word_tokenize
from collections import Counter
import time
data_posts_original  = pd.read_csv('/content/drive/MyDrive/Cognitive_Distortions_DrFarhanAli_Tawseeq/Dataset/teen_sampled_posts.csv')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Cognitive_Distortions_DrFarhanAli_Tawseeq')
import targetwords

In [ ]:
# selected_columns_data_posts_original = ['title', 'body', 'created_utc']
# reddit_posts = data_posts_original[selected_columns_data_posts_original]
n_order_unigrams = 1
n_order_bigrams = 2
n_order_trigrams = 3
n_order_quadgrams = 4
n_order_pentagrams = 5
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
covid_start = 1579478400  # UTC timestamp when COVID-19 started in singapore
covid_end = 1594598400    # UTC timestamp when COVID-19 is assumed to be finished

data_posts_original['Row No.'] = data_posts_original.index

before_covid = data_posts_original[data_posts_original['created_utc'] < covid_start]
during_covid = data_posts_original[(data_posts_original['created_utc'] >= covid_start) & (data_posts_original['created_utc'] <= covid_end)]
after_covid = data_posts_original[data_posts_original['created_utc'] > covid_end]

# Tokenizing
tokenized_before_covid_title = [word_tokenize(str(text).lower()) for text in before_covid['title']]
tokenized_during_covid_title = [word_tokenize(str(text).lower()) for text in during_covid['title']]
tokenized_after_covid_title = [word_tokenize(str(text).lower()) for text in after_covid['title']]

tokenized_before_covid_body = [word_tokenize(str(text).lower()) for text in before_covid['body']]
tokenized_during_covid_body = [word_tokenize(str(text).lower()) for text in during_covid['body']]
tokenized_after_covid_body = [word_tokenize(str(text).lower()) for text in after_covid['body']]

def process_data(tokenized_titles, tokenized_bodies, original_data):
    results_df = pd.DataFrame(columns=['Post No.'] + [distortion for distortion in targetwords.__dict__.keys() if 'target_' in distortion])
    for i, (title_tokens, body_tokens) in enumerate(zip(tokenized_titles, tokenized_bodies)):
        distortion_counts = {distortion: 0 for distortion in targetwords.__dict__.keys() if 'target_' in distortion}
        for token in title_tokens + body_tokens:  
            for distortion in distortion_counts.keys():
                if token in targetwords.__dict__[distortion]:
                    distortion_counts[distortion] += 1
        row = {'Post No.': original_data.iloc[i]['Row No.']}
        row.update(distortion_counts)
        results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)
    return results_df

before_covid_results1 = process_data(tokenized_before_covid_title, tokenized_before_covid_body, before_covid)
during_covid_results1 = process_data(tokenized_during_covid_title, tokenized_during_covid_body, during_covid)
after_covid_results1 = process_data(tokenized_after_covid_title, tokenized_after_covid_body, after_covid)

In [ ]:
before_covid_results1.to_csv('your_path', index=False)
during_covid_results1.to_csv('your_path', index=False)
after_covid_results1.to_csv('your_path', index=False)

In [ ]:
tokenized_data_posts_title = [word_tokenize(text.lower()) for text in data_posts_original['title']]
tokenized_data_posts_body = [word_tokenize(str(text).lower()) for text in data_posts_original['body']]

In [ ]:
def generate_ngrams_with_timestamps(tokens_list, n_order, timestamps):
    ngram_data = []
    for i, tokens in enumerate(tokens_list):
        ngrams_list = list(ngrams(tokens, n_order))
        for ngram in ngrams_list:
            ngram_data.append((ngram, timestamps[i], i))  
    return ngram_data


timestamps = data_posts_original['created_utc']

unigram_data_title = generate_ngrams_with_timestamps(tokenized_data_posts_title, n_order_unigrams, timestamps)
bigram_data_title = generate_ngrams_with_timestamps(tokenized_data_posts_title, n_order_bigrams, timestamps)
trigram_data_title = generate_ngrams_with_timestamps(tokenized_data_posts_title, n_order_trigrams, timestamps)
quadgram_data_title = generate_ngrams_with_timestamps(tokenized_data_posts_title, n_order_quadgrams, timestamps)
pentagram_data_title = generate_ngrams_with_timestamps(tokenized_data_posts_title, n_order_pentagrams, timestamps)

unigram_data_body = generate_ngrams_with_timestamps(tokenized_data_posts_body, n_order_unigrams, timestamps)
bigram_data_body = generate_ngrams_with_timestamps(tokenized_data_posts_body, n_order_bigrams, timestamps)
trigram_data_body = generate_ngrams_with_timestamps(tokenized_data_posts_body, n_order_trigrams, timestamps)
quadgram_data_body = generate_ngrams_with_timestamps(tokenized_data_posts_body, n_order_quadgrams, timestamps)
pentagram_data_body = generate_ngrams_with_timestamps(tokenized_data_posts_body, n_order_pentagrams, timestamps)


In [ ]:
print(np.max(data_posts_original['created_utc']))
print(np.min(data_posts_original['created_utc']))

1702371866.0
1546276524.0


In [ ]:
(1706750948- 1546274545)/604800

265.3379679232804

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

all_unigrams = unigram_data_title + unigram_data_body
all_bigrams = bigram_data_title + bigram_data_body
all_trigrams = trigram_data_title + trigram_data_body
all_quadgrams = quadgram_data_title + quadgram_data_body
all_pentagrams = pentagram_data_title + pentagram_data_body

flat_unigrams, timestamps_unigrams = zip(*all_unigrams)
flat_bigrams, timestamps_bigrams = zip(*all_bigrams)
flat_trigrams, timestamps_trigrams = zip(*all_trigrams)
flat_quadgrams, timestamps_quadgrams = zip(*all_quadgrams)
flat_pentagrams, timestamps_pentagrams = zip(*all_pentagrams)

dates_unigrams = [datetime.utcfromtimestamp(timestamp) for timestamp in timestamps_unigrams]
dates_bigrams = [datetime.utcfromtimestamp(timestamp) for timestamp in timestamps_bigrams]
dates_trigrams = [datetime.utcfromtimestamp(timestamp) for timestamp in timestamps_trigrams]
dates_quadgrams = [datetime.utcfromtimestamp(timestamp) for timestamp in timestamps_quadgrams]
dates_pentagrams = [datetime.utcfromtimestamp(timestamp) for timestamp in timestamps_pentagrams]

start_date = datetime(2018, 12, 31)
end_date = datetime(2024, 2, 4)

num_weeks = int((end_date - start_date).days / 7) + 1

weekly_counts_unigrams_list = [0] * num_weeks
# weekly_counts_bigrams_list = [0] * num_weeks
# weekly_counts_trigrams_list = [0] * num_weeks
# weekly_counts_quadgrams_list = [0] * num_weeks
# weekly_counts_pentagrams_list = [0] * num_weeks

for i, date in enumerate(dates_unigrams):
    week_number = int((date - start_date).days / 7)
    weekly_counts_unigrams_list[week_number] += 1

# for i, date in enumerate(dates_bigrams):
#     week_number = int((date - start_date).days / 7)
#     weekly_counts_bigrams_list[week_number] += 1

# for i, date in enumerate(dates_trigrams):
#     week_number = int((date - start_date).days / 7)
#     weekly_counts_trigrams_list[week_number] += 1

# for i, date in enumerate(dates_quadgrams):
#     week_number = int((date - start_date).days / 7)
#     weekly_counts_quadgrams_list[week_number] += 1

# for i, date in enumerate(dates_pentagrams):
#     week_number = int((date - start_date).days / 7)
#     weekly_counts_pentagrams_list[week_number] += 1

# plt.figure(figsize=(12, 6))
# plt.plot([start_date + timedelta(weeks=i) for i in range(num_weeks)], weekly_counts_unigrams_list, label='Unigrams', marker='o', linestyle='-')
# plt.plot([start_date + timedelta(weeks=i) for i in range(num_weeks)], weekly_counts_bigrams_list, label='Bigrams', marker='o', linestyle='-')
# plt.plot([start_date + timedelta(weeks=i) for i in range(num_weeks)], weekly_counts_trigrams_list, label='Trigrams', marker='o', linestyle='-')
# plt.plot([start_date + timedelta(weeks=i) for i in range(num_weeks)], weekly_counts_quadgrams_list, label='Quadgrams', marker='o', linestyle='-')
# plt.plot([start_date + timedelta(weeks=i) for i in range(num_weeks)], weekly_counts_pentagrams_list, label='Pentagrams', marker='o', linestyle='-')

# plt.title('Total Number of n-grams for Posts Weekly')
# plt.xlabel('Week')
# plt.ylabel('Number of n-grams')
# plt.legend()
# plt.grid(True)
# plt.show()


In [ ]:
weekly_counts_unigrams = np.array(weekly_counts_unigrams_list)
weekly_counts_unigrams

In [ ]:
weekly_counts_unigrams.shape

(266,)

In [ ]:
import time
weeks = np.zeros(267)
def count_occurrences_for_target_1gram(flat_unigrams, target_word):
    start_time = time.time()
    count = 0
    for i in range(len(flat_unigrams)):
      #current_word = str(flat_unigrams[i])
      current_word = ' '.join(map(str, flat_unigrams[i][0]))
      #print(target_word)
      if current_word.lower() == target_word.lower():
        count += 1
        index = np.floor((1706750948 - flat_unigrams[i][1])/604800)
        index = int(index)
        weeks[index] +=1
    end_time = time.time()
    return count, end_time - start_time

#for target_word in targetwords.target_dichotomous_Reasoning:
#for target_word in targetwords.target_disqualifying_the_positives:
# for target_word in targetwords.target_emotional_reasoning:
# for target_word in targetwords.target_fortune_telling:
# for target_word in targetwords.target_emotional_reasoning:
#for target_word in targetwords.target_catastrophizing:
# for target_word in targetwords.target_labeling_and_mislabeling:
# for target_word in targetwords.target_magnification_and_minimization:
# for target_word in targetwords.target_mental_filtering:
for target_word in targetwords.target_mindreading:
#for target_word in targetwords.target_overgeneralizing:
#for target_word in targetwords.target_personalizing:
# for target_word in targetwords.target_should_statements:

    count1, time_taken1 = count_occurrences_for_target_1gram(unigram_data_title, target_word)
    count2, time_taken2 = count_occurrences_for_target_1gram(bigram_data_title, target_word)
    count3, time_taken3 = count_occurrences_for_target_1gram(trigram_data_title, target_word)
    count4, time_taken4 = count_occurrences_for_target_1gram(quadgram_data_title, target_word)
    count5, time_taken5 = count_occurrences_for_target_1gram(pentagram_data_title, target_word)
    # print(f"Occurrences of the target 1-gram '{target_word}' in title: {count1+count2+count3+count4+count5}")
    # print(f"Time taken: {time_taken1+time_taken2+time_taken3+time_taken4+time_taken5} seconds")


    count6, time_taken6 = count_occurrences_for_target_1gram(unigram_data_body, target_word)
    count7, time_taken7 = count_occurrences_for_target_1gram(bigram_data_body, target_word)
    count8, time_taken8 = count_occurrences_for_target_1gram(trigram_data_body, target_word)
    count9, time_taken9 = count_occurrences_for_target_1gram(quadgram_data_body, target_word)
    count10, time_taken10 = count_occurrences_for_target_1gram(pentagram_data_body, target_word)
    print(f"Occurrences of the target 1-gram '{target_word}' in posts: {count1+count2+count3+count4+count5+count6+count7+count8+count9+count10}")
    print(f"Time taken: {time_taken1+time_taken2+time_taken3+time_taken4+time_taken5+time_taken6+time_taken7+time_taken8+time_taken9+time_taken10} seconds")
    print("---")


Occurrences of the target 1-gram 'everyone believes' in posts: 2
Time taken: 23.544708013534546 seconds
---
Occurrences of the target 1-gram 'every1 believes' in posts: 0
Time taken: 22.018723249435425 seconds
---
Occurrences of the target 1-gram 'everyone knows' in posts: 44
Time taken: 23.396789073944092 seconds
---
Occurrences of the target 1-gram 'every1 knows' in posts: 0
Time taken: 22.0704927444458 seconds
---
Occurrences of the target 1-gram 'everyone thinks' in posts: 27
Time taken: 23.141074180603027 seconds
---
Occurrences of the target 1-gram 'every1 thinks' in posts: 0
Time taken: 22.088923692703247 seconds
---
Occurrences of the target 1-gram 'everyone will believe' in posts: 0
Time taken: 22.65079426765442 seconds
---
Occurrences of the target 1-gram 'everyone'll believe' in posts: 0
Time taken: 22.906684637069702 seconds
---
Occurrences of the target 1-gram 'every1 will believe' in posts: 0
Time taken: 22.310872316360474 seconds
---
Occurrences of the target 1-gram 'eve

In [ ]:
weeks
sumw = np.sum(weeks)
print(sumw)
#week 55 is the covid start date in singapore

719.0


In [ ]:
weeks

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  3.,  6.,  7.,  2.,  3.,
        1.,  1.,  4.,  4.,  1.,  7.,  0.,  4.,  5.,  4.,  6.,  3.,  2.,
        3.,  6.,  4.,  2.,  2.,  2.,  2.,  1.,  1.,  0.,  1.,  1.,  2.,
        2.,  2.,  0.,  1.,  5.,  2.,  4.,  2.,  1.,  0.,  0.,  2.,  4.,
        1.,  4.,  1.,  5.,  4.,  1.,  0.,  2.,  1.,  2.,  5.,  3.,  4.,
        6.,  0.,  4.,  0.,  2.,  0.,  3.,  4.,  3.,  1.,  2.,  3.,  5.,
        2.,  2.,  2.,  1.,  3.,  3.,  1.,  2.,  2.,  0.,  2.,  3.,  1.,
        4.,  2.,  1.,  2.,  1.,  3.,  3.,  8.,  3.,  2.,  3.,  3.,  1.,
        7.,  6.,  2.,  2.,  7.,  5.,  2.,  3.,  4.,  1.,  2.,  1.,  1.,
        1.,  1.,  1.,  0.,  1.,  4.,  2.,  1.,  3.,  5.,  0.,  3.,  3.,
        4.,  2.,  1.,  2.,  1.,  3.,  3.,  3.,  1.,  1.,  4.,  9.,  2.,
        6.,  3.,  2.,  4.,  3.,  1.,  6.,  1.,  6.,  1.,  6.,  1.,  2.,
        2.,  0.,  5.,  5.,  1.,  1.,  3.,  3.,  6.,  6.,  6.,  1.,  1.,
        0.,  5.,  5.,  2.,  0.,  6.,  3.,  4.,  2.,  0.,  4.,  1